Тут:
1. На сьогднішню дату перевіряємо, чи є репорт за поточний місяць. Якщо його ще не має <Response [204]>,
    віднімаємо по дню від сьогодньшної дати та перевіряємо, що повертає request.
2. Формуємо {date}, щоб вставити його у API та отримуємо робочий лінк.
3. З робочого лінку отримуємо репорт на останню дату у вигляді json
4. Отримаємо актуальний список банків з їхніми МФО та полем gr_bank,що є:
    "А" - Банки з державною часткою
    "B" - Банки іноземних банківських груп
    "E" - Банки з приватним капіталом
    "F" - n/a

In [1]:
import requests
from datetime import timedelta
from datetime import date
from datetime import datetime

In [2]:
# get the last actual link from today 


def last_report(today):
    BASE_URL = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/banksfinrep?date={date}&period=m&json'
    month = today.month
    year = today.year

    if month in range(10,11,12):
        date=f"{year}{month}01"
    else:
        date=f"{year}0{month}01"
            
    url = BASE_URL.format(date=date)
    r = requests.get(url)
    return r,url


In [3]:
today=datetime.today()

In [4]:
r,_=last_report(today)
r,today.month

(<Response [204]>, 9)

In [5]:
r,url=last_report(today)
while r.status_code !=200:
        today=today-timedelta(days=1)
        r,url=last_report(today)

In [6]:
# good advice for 'while','try' :
# https://stackoverflow.com/questions/36142149/python-run-try-again-after-exception-caught-and-worked-out/36142210
while True:
    try:
        r,url=last_report(today)
        while r.status_code !=200:
            today=today-timedelta(days=1)
            r,url=last_report(today)# try code
        break # quit the loop if successful
    except:
        print("No connection with API")# error handling



In [7]:
today,r

(datetime.datetime(2020, 8, 31, 12, 0, 8, 983311), <Response [200]>)

In [8]:
url

'https://bank.gov.ua/NBUStatService/v1/statdirectory/banksfinrep?date=20200801&period=m&json'

In [9]:
r = requests.get(url)
source=r.json()

In [10]:
name_pair = [(i['fullname'],i['mfo'],i['gr_bank']) for i in source]

In [11]:
banks = set(name_pair)

In [12]:
type(list(banks))

list

In [13]:
source[0]

{'dt': '20200801',
 'txt': 'Активи - Чисті активи, усього',
 'txten': 'Assets - Net assets, Total',
 'id_api': 'BS1_AssetsNet',
 'leveli': 2,
 'parent': 'BS1_AssetsTotal',
 'freq': 'M',
 'nkb': '305',
 'mfo': '307123',
 'fullname': 'ПАТ "БАНК ВОСТОК"',
 'gr_bank': 'E',
 'r034': '1',
 'value': 6771.3352,
 'tzep': 'T070_7'}

In [16]:
leveli_1=list(set([(i['txt'],i['txten'],i['id_api']) for i in source if i['leveli']==1]))

In [18]:
len(leveli_1)

7

In [41]:
leveli_1[0][2],leveli_1[0][0]

('BS1_ProfitLossAfterTax',
 'Доходи і витрати - Прибуток/(збиток) після оподаткування')

In [29]:
leveli_1[1][2],leveli_1[1][0]

('BS1_AssetsTotal', 'Активи - Усього активів')

In [48]:
BS1_AssetsTotal = list(set([(i['txt'],i['txten'],i['id_api']) for i in source if i['parent']=='BS1_AssetsTotal']))

In [30]:
leveli_1[2][2],leveli_1[2][0]
#BS1_Capital_Total

('BS1_CapitalTotal', 'Капітал - Усього власного капіталу')

In [49]:
BS1_Capital_Total = list(set([(i['txt'],i['txten'],i['id_api']) for i in source if i['parent']=='BS1_Capital_Total']))

In [32]:
leveli_1[3][2],leveli_1[3][0]

('BS1_LiabTotal', "Зобов'язання - Усього зобов’язань")

In [51]:
BS1_LiabTotal=list(set([(i['txt'],i['txten'],i['id_api']) for i in source if i['parent']=='BS1_LiabTotal']))

In [34]:
leveli_1[4][2],leveli_1[4][0]

('BS1_ProfitLossBeforTax',
 'Доходи і витрати - Прибуток/(збиток) до оподаткування')

In [52]:
BS1_ProfitLossBeforTax = list(set([(i['txt'],i['txten'],i['id_api']) for i in source if i['parent']=='BS1_ProfitLossBeforTax']))

In [36]:
leveli_1[5][2],leveli_1[5][0]

('BS1_ExpenTotal', 'Доходи і витрати - Всього витрат')

In [53]:
BS1_ExpenTotal = list(set([(i['txt'],i['txten'],i['id_api']) for i in source if i['parent']=='BS1_ExpenTotal']))

In [38]:
leveli_1[6][2],leveli_1[6][0]

('BS1_IncomeTotal', 'Доходи і витрати - Всього доходів')

In [55]:
BS1_IncomeTotal = list(set([(i['txt'],i['txten'],i['id_api']) for i in source if i['parent']=='BS1_IncomeTotal']))

In [58]:
BS1_AssetsTotal


[('Активи - Provisions, total',
  'Assets - Net assets - Total',
  'BS1_AssetsProvTotal'),
 ('Активи - Інвестиційна нерухомість',
  'Assets - Investment property',
  'BS1_AssetsInvestProp'),
 ('Активи - Цінні папери, які обліковуються за справедливою вартістю через інший сукупний дохід',
  'Assets - Securities at fair value through other comprehensive income',
  'BS1_AssetsSecurFVOCI'),
 ('Активи - ОВДП, що рефінансуються НБУ',
  'Assets - IGLB refinanced by NBU',
  'BS1_AssetsIGLBRefinNBU'),
 ('Активи - Інші активи', 'Assets - Other assets', 'BS1_AssetsOther'),
 ('Активи - Чисті активи, усього',
  'Assets - Net assets, Total',
  'BS1_AssetsNet'),
 ('Активи - Відстрочений податковий актив',
  'Assets - Deferred tax asset',
  'BS1_AssetsDeferTax'),
 ('Активи - Цінні папери, які обліковуються за амортизованою собівартістю',
  'Assets - Securities at amortised cost',
  'BS1_AssetsSecurAC'),
 ('Активи - Інвестиції в асоційовані та дочірні компанії',
  'Assets - Investments in associates an